# Training  and testing LeNet using caffe
Jupyter notebook을 사용하여 Linux Shell 명령어로 caffe를 사용하는 방법에 대해서 배웁니다.

본 예제는 크게 두 Part로 나누어져 있습니다.
1. Caffe 명령어를 통해 모델을 학습하는 방법
2. 학습된 모델의 test data에 대한 Classification 성능을 측정하는 방법

### 0. Basic Usage

* Jupyter Notebook에서 Linux Shell 명령어를 사용하기 위해서는 명령앞에 !를 붙여서 실행하시면 됩니다.

In [5]:
!pwd

/root/dlworkshop/2_caffe_intro


In [6]:
!ls ./lenet

create_mnist.sh		      lenet_iter_5000.caffemodel   mnist.pyc
lenet.prototxt		      lenet_iter_5000.solverstate  mnist_test_lmdb
lenet5.png		      lenet_solver.prototxt	   mnist_train_lmdb
lenet_iter_10000.caffemodel   lenet_train_test.prototxt
lenet_iter_10000.solverstate  mnist.py


## 1. Training LeNet 5

<img src=./lenet/lenet5.png>

### 1-1. Convert MNIST dataset

본 예제에서 사용할 데이터셋은 MNIST로 0~9까지의 숫자 이미지와 이미지에 해당하는 Label을 포함합니다.
MNIST 데이터셋에 대해 자세한 내용은 http://yann.lecun.com/exdb/mnist 를 참고해 주세요.

Raw MNIST dataset은 제공해드린 docker 이미지상에 이미 다운로드 되어 있으며, 해당 위치는 ./caffe/data/mnist 입니다.
하지만 Raw MNIST dataset의 형식은 caffe에서 지원되지 않기에 caffe에서 가장 많이 쓰는 LMDB형식으로 변환되어야 합니다.
변환에 필요한 코드는 이미 작성되어 있으며 ./lenet/create_mnist.sh 을 실행하여 변환합니다.

스크립트 실행시 발생하는 libdc1394 error는 docker의 driver상의 오류로 무시하셔도 됩니다

In [7]:
!sh ./lenet/create_mnist.sh

Creating lmdb...
libdc1394 error: Failed to initialize libdc1394
libdc1394 error: Failed to initialize libdc1394
Done.


변환이 완료된 후, !ls ./lenet 명령어를 통해 mnist_test_lmdb와 mnist_train_lmdb가 정상적으로 생성되었음을 확인하실 수 있습니다.

In [8]:
!ls ./lenet

create_mnist.sh		      lenet_iter_5000.caffemodel   mnist.pyc
lenet.prototxt		      lenet_iter_5000.solverstate  mnist_test_lmdb
lenet5.png		      lenet_solver.prototxt	   mnist_train_lmdb
lenet_iter_10000.caffemodel   lenet_train_test.prototxt
lenet_iter_10000.solverstate  mnist.py


## 1-2. Define model architecture

LeNet5의 모델은 ./lenet/lenet_train_test.prototxt에 정의되어 있습니다. 

cat 명령어는 Linux에서 text파일을 모니터에 출력해주는 함수로 prototxt 파일을 읽어올 수 있습니다.
명령어 실행 결과를 참고하여 각 layer가 요구하는 parameter를 참고하세요.

prototxt는 jupyter notebook 상에서 해당 파일을 클릭하는 것으로 수정하실 수 있습니다.

In [9]:
!cat ./lenet/lenet_train_test.prototxt

name: "LeNet"
layer {
  name: "mnist"
  type: "Data"
  top: "data"
  top: "label"
  include {
    phase: TRAIN
  }
  transform_param {
    scale: 0.00390625
  }
  data_param {
    source: "./lenet/mnist_train_lmdb"
    batch_size: 64
    backend: LMDB
  }
}
layer {
  name: "mnist"
  type: "Data"
  top: "data"
  top: "label"
  include {
    phase: TEST
  }
  transform_param {
    scale: 0.00390625
  }
  data_param {
    source: "./lenet/mnist_test_lmdb"
    batch_size: 100
    backend: LMDB
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  param {
    lr_mult: 1
  }
  param {
    lr_mult: 2
  }
  convolution_param {
    num_output: 20
    kernel_size: 5
    stride: 1
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 2
    stride: 2
  }
}
layer {
  name: "conv2"
  type: "Convolutio

## 1-3. Define solver

정의된 Network를 학습하기 위해서 Solver를 prototxt형식으로 정의합니다. 

Solver hyperparameters
- Solver: SGD (Stochastic gradient descent)
- Base Learning Rate: 0.01 (Staring learning rate)
- Momemtum: 0.9 (SGD parameter)
- weight_decay: 0.005
- lr_policy: inv 
- Gamma: 0.0001
- Power: 0.75
- solver_mode: CPU  

Display param
- display: 100 (training 100 batch 마다 Loss 출력)
- test_iter: 100 (validation data시 100개의 batch를 테스트)
- test_interval: 500 (training 500 batch 마다 validation data test)
- max_iter: 10000 (10000 batch training 도달시 종료)

Snapshot param
- snapshot: 5000 (5000 batch training 마다 모델 저장)
- snapshot_prefix: ./lenet/lenet (모델 저장 위치와 이름에 대한 prefix)

위의 내용을 caffe의 prototxt 형식으로 나타내면 아래와 같습니다.

In [10]:
!cat ./lenet/lenet_solver.prototxt

# The train/test net protocol buffer definition
net: "./lenet/lenet_train_test.prototxt"
# test_iter specifies how many forward passes the test should carry out.
# In the case of MNIST, we have test batch size 100 and 100 test iterations,
# covering the full 10,000 testing images.
test_iter: 100
# Carry out testing every 500 training iterations.
test_interval: 500
# The base learning rate, momentum and the weight decay of the network.
base_lr: 0.01
momentum: 0.9
weight_decay: 0.0005
# The learning rate policy
lr_policy: "inv"
gamma: 0.0001
power: 0.75
# Display every 100 iterations
display: 100
# The maximum number of iterations
max_iter: 10000
# snapshot intermediate results
snapshot: 5000
snapshot_prefix: "./lenet/lenet"
# solver mode: CPU or GPU
solver_mode: CPU


## 1-4. Train LeNet5

빌드된 Caffe의 실행파일은 ./caffe/build/tools/caffe에 존재합니다.

caffe 실행파일의 사용법(commands & arguments)을 보기 위해서 아래의 명령어를 실행합니다.

In [12]:
!./caffe/build/tools/caffe

libdc1394 error: Failed to initialize libdc1394
caffe: command line brew
usage: caffe <command> <args>

commands:
  train           train or finetune a model
  test            score a model
  device_query    show GPU diagnostic information
  time            benchmark model execution time

  Flags from /root/caffe/tools/caffe.cpp:
    -gpu (Optional; run in GPU mode on given device IDs separated by ','.Use
      '-gpu all' to run on all available GPUs. The effective training batch
      size is multiplied by the number of devices.) type: string default: ""
    -iterations (The number of iterations to run.) type: int32 default: 50
    -model (The model definition protocol buffer text file.) type: string
      default: ""
    -sighup_effect (Optional; action to take when a SIGHUP signal is received:
      snapshot, stop or none.) type: string default: "snapshot"
    -sigint_effect (Optional; action to take when a SIGINT signal is received:
      snapshot, stop or none.) type: string defau

가장 간단하게는 ./caffe/build/tools/caffe train --solver=/path/to/solver/file.prototxt로 정의된 모델을 정의된 solver로 학습시킬 수 있습니다.

Desktop에서 모델을 전부 학습하기 위해서는 약 10여분이 소요됩니다. (노트북의 경우 15분 이상이 소요될 수 있습니다.)

#### 중단을 원하시면 Jupyter notebook 상단의 Kernel 탭 클릭 후 Interrupt를 클릭해주세요.
#### Shell 상에서 중단 (Jupyter notebook 상의 kernel interrupt)이 될 경우 중단 시점에서의 snapshot이 자동저장됩니다.

### Jupyter notebook 상의 ln [\*] 표시는 현재 처리중임을 의미하며, 완료가 되면 \* 기호는 숫자로 변경됩니다.

In [13]:
!./caffe/build/tools/caffe train --solver=./lenet/lenet_solver.prototxt

libdc1394 error: Failed to initialize libdc1394
I0427 14:47:38.377068  1084 caffe.cpp:178] Use CPU.
I0427 14:47:38.377388  1084 solver.cpp:48] Initializing solver from parameters: 
test_iter: 100
test_interval: 500
base_lr: 0.01
display: 100
max_iter: 10000
lr_policy: "inv"
gamma: 0.0001
power: 0.75
momentum: 0.9
weight_decay: 0.0005
snapshot: 5000
snapshot_prefix: "./lenet/lenet"
solver_mode: CPU
net: "./lenet/lenet_train_test.prototxt"
I0427 14:47:38.377507  1084 solver.cpp:91] Creating training net from net file: ./lenet/lenet_train_test.prototxt
I0427 14:47:38.377833  1084 net.cpp:313] The NetState phase (0) differed from the phase (1) specified by a rule in layer mnist
I0427 14:47:38.377861  1084 net.cpp:313] The NetState phase (0) differed from the phase (1) specified by a rule in layer accuracy
I0427 14:47:38.377943  1084 net.cpp:49] Initializing net from parameters: 
name: "LeNet"
state {
  phase: TRAIN
}
layer {
  name: "mnist"
  type: "Data"
  top: "data"
  top: "label"
  inc

Caffe에서는 예기치 않은 이유로 학습이 중단되었을 경우 저장된 지점(.solverstate 파일)이 있다면 이어서 학습이 가능합니다. (Snapshot resume 기능)

이미 5000번까지 학습된 Solver state가 저장되어 있으므로 아래의 명령어를 통해 iteration 5000번부터 이어서 학습이 가능합니다.

In [14]:
!./caffe/build/tools/caffe train --solver=./lenet/lenet_solver.prototxt --snapshot=./lenet/lenet_iter_5000.solverstate

libdc1394 error: Failed to initialize libdc1394
I0427 14:52:17.499214  1090 caffe.cpp:178] Use CPU.
I0427 14:52:17.499536  1090 solver.cpp:48] Initializing solver from parameters: 
test_iter: 100
test_interval: 500
base_lr: 0.01
display: 100
max_iter: 10000
lr_policy: "inv"
gamma: 0.0001
power: 0.75
momentum: 0.9
weight_decay: 0.0005
snapshot: 5000
snapshot_prefix: "./lenet/lenet"
solver_mode: CPU
net: "./lenet/lenet_train_test.prototxt"
I0427 14:52:17.499645  1090 solver.cpp:91] Creating training net from net file: ./lenet/lenet_train_test.prototxt
I0427 14:52:17.499848  1090 net.cpp:313] The NetState phase (0) differed from the phase (1) specified by a rule in layer mnist
I0427 14:52:17.499872  1090 net.cpp:313] The NetState phase (0) differed from the phase (1) specified by a rule in layer accuracy
I0427 14:52:17.499933  1090 net.cpp:49] Initializing net from parameters: 
name: "LeNet"
state {
  phase: TRAIN
}
layer {
  name: "mnist"
  type: "Data"
  top: "data"
  top: "label"
  inc

본 과정이 모두 완료되면 10000 minibatch의 학습 후 Test dataset에서 약 99.1%의 인식 성능을 갖는 모델(./lenet/lenet_iter_10000.caffemodel)을 얻을 수 있습니다.


In [ ]:
!ls ./lenet

## 2. Testing LeNet 5

In [15]:
!./caffe/build/tools/caffe test --model=./lenet/lenet_train_test.prototxt --weights=./lenet/lenet_iter_10000.caffemodel --iterations=100

libdc1394 error: Failed to initialize libdc1394
I0427 15:17:46.006711  1096 caffe.cpp:246] Use CPU.
I0427 15:17:46.008276  1096 net.cpp:313] The NetState phase (1) differed from the phase (0) specified by a rule in layer mnist
I0427 15:17:46.008383  1096 net.cpp:49] Initializing net from parameters: 
name: "LeNet"
state {
  phase: TEST
}
layer {
  name: "mnist"
  type: "Data"
  top: "data"
  top: "label"
  include {
    phase: TEST
  }
  transform_param {
    scale: 0.00390625
  }
  data_param {
    source: "./lenet/mnist_test_lmdb"
    batch_size: 100
    backend: LMDB
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  param {
    lr_mult: 1
  }
  param {
    lr_mult: 2
  }
  convolution_param {
    num_output: 20
    kernel_size: 5
    stride: 1
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX


In [22]:
%%writefile aa.prototxt
adfasdfasdfasd

Overwriting aa.prototxt


create_mnist.sh		      lenet_iter_5000.caffemodel   mnist_test_lmdb
lenet5.png		      lenet_iter_5000.solverstate  mnist_train_lmdb
lenet_iter_10000.caffemodel   lenet_solver.prototxt
lenet_iter_10000.solverstate  lenet_train_test.prototxt
